##### IMPORT

In [2]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os

##### ENV VARIABLES

In [3]:
load_dotenv()
api_key = os.getenv("API_KEY")
user = os.getenv("DB_USER")
pswd = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
name = os.getenv("DB_NAME")

##### MODEL DEFINITION

In [4]:
llm = GoogleGenerativeAI(google_api_key=api_key, model = "gemini-2.0-flash", temperature=0.2)
poem = llm("write a poem on my love for dosa")
print(poem)

C:\Users\layas\AppData\Local\Temp\ipykernel_3480\1783157393.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  poem = llm("write a poem on my love for dosa")


The griddle sings a sizzling song,
A promise whispered, all night long.
Of golden crisp and airy grace,
A smile upon my hungry face.

The batter swirls, a milky dream,
Upon the heat, a vibrant gleam.
It spreads and thins, a perfect round,
The best breakfast that can be found.

From plain and simple, pure delight,
To masala's fiery, spiced might.
With potato filling, rich and bold,
A story in each fold unfolds.

The sambar steams, a tangy brew,
Coconut chutney, fresh and new.
A dip, a tear, a blissful sigh,
As flavors dance and reach the sky.

Oh, dosa, you are more than food,
A comfort, understood.
A taste of home, a warm embrace,
A smile that time cannot erase.

From street side stalls to fancy fare,
My love for you beyond compare.
So let the griddle sing its tune,
I'll be there waiting, very soon.


##### MYSQL CONNECTION

In [5]:
from langchain.utilities import SQLDatabase
db = SQLDatabase.from_uri(f"mysql+pymysql://{user}:{pswd}@{host}/{name}", sample_rows_in_table_info = 3)


In [6]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
21	1	10.00
22	2	15.00
23	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	st

##### DIRECT MYSQL QUERY

In [7]:
import pymysql

# Connect to your database
connection = pymysql.connect(
    host=host,
    user=user,
    password=pswd,
    database=name
)

try:
    with connection.cursor() as cursor:
        # Execute the query without markdown formatting
        sql = '''SELECT sum(a.total_amount*((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue
        FROM (
            SELECT sum(price*stock_quantity) AS total_amount, t_shirt_id 
            FROM t_shirts 
            WHERE brand = 'Levi'
            GROUP BY t_shirt_id
            )
        a LEFT JOIN discounts 
        ON a.t_shirt_id = discounts.t_shirt_id

            '''
        cursor.execute(sql)
        result = cursor.fetchall()
        print(result)
finally:
    connection.close()


((Decimal('20107.100000'),),)


##### SQL QUERY GENERATOR

In [8]:
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

# Create a custom tool that extends QuerySQLDatabaseTool to clean queries
class CleanQuerySQLDatabaseTool(QuerySQLDatabaseTool):
    def _run(self, query: str, **kwargs):
        # Remove markdown code block formatting if present
        if query.startswith("```"):
            # Extract the query from the markdown code block
            lines = query.split("\n")
            # Remove the first line (```sql) and the last line (```
            clean_lines = lines[1:-1] if lines[-1].strip() == "```" else lines[1:]
            query = "\n".join(clean_lines)
            print ("SQL QUERY:", query)
        return super()._run(query, **kwargs)

# Create the SQL query generation chain
write_query = create_sql_query_chain(llm, db)

# Create the custom SQL query execution tool
execute_query = CleanQuerySQLDatabaseTool(db=db)

# Chain them together
chain = write_query | execute_query

In [9]:
# Wrong
# Execute the chain with your question
qns1 = chain.invoke({"question": "How many t-shirts do we have left for nike in extra small size and white color?"})

SQL QUERY: SELECT
  `stock_quantity`
FROM t_shirts
WHERE
  `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'
LIMIT 5;


In [10]:
# Correct
# Execute the chain with your question
qns2 = chain.invoke({"question": "How much is the price of the inventory for all small size t-shirts?"})

SQL QUERY: SELECT SUM(`price` * `stock_quantity`) AS inventory_value FROM t_shirts WHERE `size` = 'S'


In [11]:
# Wrong
# Execute the chain with your question
qns3 = chain.invoke({"question": "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"})

SQL QUERY: SELECT
  SUM(
    (
      `t_shirts`.`price` * (
        1 - `discounts`.`pct_discount` / 100
      )
    ) * `t_shirts`.`stock_quantity`
  ) AS revenue
FROM `t_shirts`
JOIN `discounts`
  ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id`
WHERE
  `t_shirts`.`brand` = 'Levi';


In [12]:
qns3

"[(Decimal('7115.100000'),)]"

In [13]:
# Correct
# Execute the chain with your question
qns4 = chain.invoke({"question": "How many white color Levi's t shirts we have available?"})
qns4

SQL QUERY: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `color` = 'White' AND `brand` = 'Levi'


"[(Decimal('80'),)]"

In [14]:
# Correct
# Execute the chain with your question
qns5 = chain.invoke({"question": "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?"})
qns5

SQL QUERY: SELECT SUM(`price` * `stock_quantity`) AS total_revenue
FROM `t_shirts`
WHERE `brand` = 'Levi';


"[(Decimal('22109'),)]"

In [15]:
# Correct
# Execute the chain with your question
qns6 = chain.invoke({"question": "How many Levi's t shirts we have available?"})
qns6

SQL QUERY: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi'


"[(Decimal('779'),)]"

In [16]:
# Wrong
# Execute the chain with your question
qns7 = chain.invoke({"question": "Which t-shirt gives the highest discount [in terms of money]"})
qns7

SQL QUERY: SELECT
  `t`.`t_shirt_id`
FROM t_shirts AS `t`
JOIN discounts AS `d`
  ON `t`.`t_shirt_id` = `d`.`t_shirt_id`
ORDER BY
  `t`.`price` * `d`.`pct_discount` DESC
LIMIT 1;


'[(10,)]'

In [17]:
# Execute the chain with your question
qns7 = chain.invoke({"question": "Execute this query: SELECT sum(a.total_amount*((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue FROM ( SELECT sum(price*stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id ) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id"})
qns7

SQL QUERY: SELECT
  sum(a.total_amount * ((
    100 - COALESCE(discounts.pct_discount, 0)
  ) / 100)) AS total_revenue
FROM (
  SELECT
    sum(`price` * `stock_quantity`) AS total_amount,
    `t_shirt_id`
  FROM t_shirts
  WHERE
    `brand` = 'Levi'
  GROUP BY
    `t_shirt_id`
) AS a
LEFT JOIN discounts
  ON a.`t_shirt_id` = discounts.`t_shirt_id`;


"[(Decimal('20107.100000'),)]"

In [18]:
# Execute the chain with your question
qns8 = chain.invoke({"question": "Execute this query: SELECT MAX(price*pct_discount/100) AS discount_amount FROM t_shirts t JOIN discounts d ON t.t_shirt_id = d.t_shirt_id;"})
qns8

SQL QUERY: SELECT MAX(`price`*`pct_discount`/100) AS discount_amount FROM t_shirts t JOIN discounts d ON t.`t_shirt_id` = d.`t_shirt_id`;


"[(Decimal('12.600000'),)]"

##### FEW SHOT LEARNING

In [19]:
few_shots = [
    {
        'Question': "How many t-shirts do we have left for nike in extra small size and white color?",
        'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White';",
        'SQLResult': "Result of the SQL Query",
        'Answer':qns1
    },
    {
        'Question': "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
        'SQLQuery': "SELECT sum(a.total_amount*((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue FROM ( SELECT sum(price*stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id ) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id",
        'SQLResult': "Result of the SQL Query",
        'Answer':qns7
    },
    {
        'Question': "What is the highest discount in terms of money that is given?",
        'SQLQuery': "SELECT MAX(price*pct_discount/100) AS discount_amount FROM t_shirts t JOIN discounts d ON t.t_shirt_id = d.t_shirt_id;",
        'SQLResult': "Result of the SQL Query",
        'Answer':qns8
    },

]

In [39]:
qns8

"[(Decimal('12.600000'),)]"

##### EMBEDDINGS

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

C:\Users\layas\AppData\Local\Temp\ipykernel_3480\282906811.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
c:\Users\layas\Desktop\Laya\Project Files\GenAI_Tshirt_Sales\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [23]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_texts(to_vectorize, embedding = embeddings, metadatas = few_shots)

In [24]:
from langchain.prompts import SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vectorstore,
    k=2,
)

In [25]:
example_selector.select_examples({"Question":"How many Adidas T shirts I have left in my store?"})

[{'SQLResult': 'Result of the SQL Query',
  'Answer': '[(11,)]',
  'Question': 'How many t-shirts do we have left for nike in extra small size and white color?',
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White';"},
 {'Answer': "[(Decimal('20107.100000'),)]",
  'SQLQuery': "SELECT sum(a.total_amount*((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue FROM ( SELECT sum(price*stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id ) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id",
  'Question': "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?",
  'SQLResult': 'Result of the SQL Query'}]

##### PROMPT TEMPLATE

In [26]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [27]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [28]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [29]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [30]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables = ["input", "table_info", "top_k"],
)

##### NEW CHAIN

In [31]:
write_query_new = create_sql_query_chain(llm, db, prompt = few_shot_prompt)

# Create the custom SQL query execution tool
execute_query_new = CleanQuerySQLDatabaseTool(db=db)

# Chain them together
new_chain = write_query_new | execute_query_new

In [32]:
# Execute the chain with your question
qns1_new = new_chain.invoke({"question": "How many t-shirts do we have left for nike in extra small size and white color?"})

SQL QUERY: SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'


In [33]:
qns1_new

"[(Decimal('11'),)]"

In [34]:
# Wrong
# Execute the chain with your question
qns3_new = new_chain.invoke({"question": "If we have to sell all the Levi's T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"})
qns3_new

SQL QUERY: SELECT
  SUM(a.total_amount * (
    (
      100 - COALESCE(discounts.pct_discount, 0)
    ) / 100
  )) AS total_revenue
FROM (
  SELECT
    SUM(price * stock_quantity) AS total_amount,
    t_shirt_id
  FROM t_shirts
  WHERE
    brand = 'Levi'
  GROUP BY
    t_shirt_id
) a
LEFT JOIN discounts
  ON a.t_shirt_id = discounts.t_shirt_id;


"[(Decimal('20107.100000'),)]"

In [35]:
# Wrong
# Execute the chain with your question
qns7_new = new_chain.invoke({"question": "Which t-shirt gives the highest discount [in terms of money]"})
qns7_new

SQL QUERY: SELECT
  `t_shirt_id`
FROM t_shirts t
JOIN discounts d
  ON t.t_shirt_id = d.t_shirt_id
ORDER BY
  price * pct_discount / 100 DESC
LIMIT 1;


'Error: (pymysql.err.OperationalError) (1052, "Column \'t_shirt_id\' in field list is ambiguous")\n[SQL: SELECT\n  `t_shirt_id`\nFROM t_shirts t\nJOIN discounts d\n  ON t.t_shirt_id = d.t_shirt_id\nORDER BY\n  price * pct_discount / 100 DESC\nLIMIT 1;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

##### ROUGH AREA

In [36]:
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


# Create the SQL query generation chain
write_query = create_sql_query_chain(llm, db)

# Create the SQL query execution tool
execute_query = QuerySQLDatabaseTool(db=db)

# Chain them together
chain = write_query | execute_query


# Execute the chain with your question
result = chain.invoke({"question": "How many t-shirts do we have left for nike in extra small size and white color?"})

print(result)


Error: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT\n  `stock_quantity`\nFROM t_shirts\nWHERE\n  `brand` = 'Nike' AND `siz' at line 1")
[SQL: ```sql
SELECT
  `stock_quantity`
FROM t_shirts
WHERE
  `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [37]:
# from langchain_experimental.sql import SQLDatabaseChain
# db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker = True)
# db_chain | RunnableLambda(lambda x:x.replace('```','').replace('sql',''))
# qns1 = db_chain.invoke("How many t-shirts do we have left for nike in extra small size and white color?")


from langchain_core.runnables import RunnableLambda
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

chain = create_sql_query_chain(llm, db)
chain | RunnableLambda(lambda x:x.replace('```','').replace('sql',''))
qns1 = chain.invoke({"question": "How many t-shirts do we have left for nike in extra small size and white color?"})

